In [4]:
#easy clustering
import pandas as pd
import numpy as np
import sys
import nbimporter
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrameReader
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col
from pyspark.mllib.linalg import Vectors

from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'

#%load_ext autotime
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [5]:
####FUNCTION AREA###########

#get workout info for workoutid
def get_rdd_workout_from_workoutid(workoutid, df_workouts):
    filtered_df = df_workouts.filter(df_workouts['workoutid']==workoutid)
    return filtered_df
    
'''
##given user id, returns dataframe of workouts by that user
##NOT DONE
def get_workouts_for_userid(userid, df_workout):
    #get workoutIds done by this user
    workout_ids = df_workout\
        .filter(df_workout['userid']==userid)\
        .select('workoutid').flatMap(lambda x:x).collect()
    print workout_ids
    df = df_sport.where(col("workoutid").isin(workout_ids))
    print df.show()
    print df.count()
    return userid
'''

#get workoutids for user
def get_rdd_workoutids_for_userid(userid, df_workouts):
    filtered_df = df_workouts.filter(df_workouts['id']==userid)
    workoutids = filtered_df.select('workoutid').flatMap(list)
    return workoutids

#convert type of a column of a df
def convertColumn(df, name, new_type):
    df_1 = df.withColumnRenamed(name, "swap")
    return df_1.withColumn(name, df_1["swap"].cast(new_type)).drop("swap")

# #fills 
# def fill_na_with_values(dict){
    

In [6]:
url = 'postgresql://34.199.246.37:5432/endomondo'
properties = {'user':'endomondo', 'password':'End0m0ndo'}

df_workouts = DataFrameReader(sqlContext).jdbc(
   url='jdbc:%s' % url, table='public.run_sub_with_hr', properties=properties
)

df_users = DataFrameReader(sqlContext).jdbc(
   url='jdbc:%s' % url, table='public.run_sub_with_hr_users', properties=properties
)


Py4JJavaError: An error occurred while calling o49.jdbc.
: org.postgresql.util.PSQLException: The connection attempt failed.
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:272)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:52)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:216)
	at org.postgresql.Driver.makeConnection(Driver.java:404)
	at org.postgresql.Driver.connect(Driver.java:272)
	at java.sql.DriverManager.getConnection(DriverManager.java:664)
	at java.sql.DriverManager.getConnection(DriverManager.java:208)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$$anonfun$getConnector$1.apply(JDBCRDD.scala:188)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$$anonfun$getConnector$1.apply(JDBCRDD.scala:181)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:121)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation.<init>(JD

In [4]:
df_workouts.show()

NameError: name 'df_workouts' is not defined

In [15]:
#make each row in the dataframe into feature vector for clustering

vecAssembler = VectorAssembler(inputCols=['altitude',\
 'heart_rate',\
 'latitude',\
 'longitude',\
 'speed'], outputCol="features")
final_df = vecAssembler.transform(df_workouts.na.fill(0))
final_df = final_df.select('features')




In [16]:
#make rows into "dense vectors" for clustering
final_df1 = final_df.rdd\
    .map(lambda row : Vectors.dense([item for item in row]))
final_df1.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/momori/app/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/Users/momori/app/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/momori/app/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-16-332c7b30138c>", line 2, in <lambda>
  File "/Users/momori/app/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/mllib/linalg/__i